In [1]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import classification_report

In [2]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/curtailment_target_features.csv', sep = ';', index_col=0)

In [4]:
# Split the entire dataset into training and test sets
cutoff_time = "2023-01-01"
train = df[df.index < cutoff_time]
test = df[df.index >= cutoff_time]
X_train = train.drop('redispatch', axis = 1)
y_train = train['redispatch']
X_test = test.drop('redispatch', axis = 1)
y_test = test['redispatch']

In [5]:
# preprocessing pipelines
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

smote = SMOTE(random_state=13)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed, y_train_preprocessed = smote.fit_resample(X_train_preprocessed, y_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [13]:
# Define number of epochs and learning rate decay
N_TRAIN = len(X_train_preprocessed)
EPOCHS = 50
BATCH_SIZE = 32
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH,
    decay_rate=1,
    staircase=False)

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')

# define model
def dl_bin_class_model():
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_dim=12),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')])

      # Compile the model with custom optimizer and metrics
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall()])
    return model

# model
model = dl_bin_class_model()


# Train model
with tf.device('/device:GPU:0'):
  history = model.fit(X_train_preprocessed,
                    y_train_preprocessed,
                    validation_split=0.2,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1)

Epoch 1/50
4967/4967 [==============================] - 20s 3ms/step - loss: 0.3723 - accuracy: 0.8244 - recall: 0.7801 - val_loss: 0.4037 - val_accuracy: 0.8378 - val_recall: 0.8378
Epoch 2/50
4967/4967 [==============================] - 17s 3ms/step - loss: 0.2504 - accuracy: 0.8939 - recall: 0.8829 - val_loss: 0.2368 - val_accuracy: 0.9269 - val_recall: 0.9269
Epoch 3/50
4967/4967 [==============================] - 17s 3ms/step - loss: 0.2061 - accuracy: 0.9158 - recall: 0.9121 - val_loss: 0.2254 - val_accuracy: 0.9236 - val_recall: 0.9236
Epoch 4/50
4967/4967 [==============================] - 17s 3ms/step - loss: 0.1825 - accuracy: 0.9285 - recall: 0.9280 - val_loss: 0.2455 - val_accuracy: 0.9140 - val_recall: 0.9140
Epoch 5/50
4967/4967 [==============================] - 17s 3ms/step - loss: 0.1669 - accuracy: 0.9355 - recall: 0.9359 - val_loss: 0.1614 - val_accuracy: 0.9488 - val_recall: 0.9488
Epoch 6/50
4967/4967 [==============================] - 17s 3ms/step - loss: 0.1545 -

In [ ]:
'''# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')'''


In [15]:
# Assuming you have already trained your model and obtained predictions on the test data
y_pred = model.predict(X_test_preprocessed)

threshold = 0.5  # Adjust the threshold as needed
y_pred_binary = (y_pred >= threshold).astype(int)

# Print classification report
print(classification_report(y_test, y_pred_binary))

1093/1093 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

         0.0       0.93      0.89      0.91     32472
         1.0       0.09      0.14      0.11      2484

    accuracy                           0.84     34956
   macro avg       0.51      0.52      0.51     34956
weighted avg       0.87      0.84      0.85     34956



In [39]:
# Define your deep learning model
model = Sequential([
    Dense(64, activation='relu', input_dim=12),  # Example input_dim
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Example output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['recall'])

# Train the model
model.fit(X_train_preprocessed, y_train_preprocessed, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Make predictions (optional)
predictions = model.predict(X_test_preprocessed)

Epoch 1/10
4967/4967 [==============================] - 15s 3ms/step - loss: 0.4285 - accuracy: 0.7931 - val_loss: 0.5099 - val_accuracy: 0.7659
Epoch 2/10
4967/4967 [==============================] - 12s 2ms/step - loss: 0.3371 - accuracy: 0.8475 - val_loss: 0.3326 - val_accuracy: 0.8813
Epoch 3/10
4967/4967 [==============================] - 13s 3ms/step - loss: 0.2945 - accuracy: 0.8728 - val_loss: 0.3802 - val_accuracy: 0.8529
Epoch 4/10
4967/4967 [==============================] - 10s 2ms/step - loss: 0.2676 - accuracy: 0.8873 - val_loss: 0.3962 - val_accuracy: 0.8365
Epoch 5/10
4967/4967 [==============================] - 9s 2ms/step - loss: 0.2486 - accuracy: 0.8967 - val_loss: 0.3867 - val_accuracy: 0.8401
Epoch 6/10
4967/4967 [==============================] - 11s 2ms/step - loss: 0.2346 - accuracy: 0.9041 - val_loss: 0.2775 - val_accuracy: 0.9042
Epoch 7/10
4967/4967 [==============================] - 13s 3ms/step - loss: 0.2228 - accuracy: 0.9107 - val_loss: 0.2123 - val_acc